In [1]:
import pandas as pd
import os
import hashlib
import xls_form_generator as xfg
import clinical_objects as cobj
import clinical_algorithm as calgo
import calculations as calc

<h1><span style="color:#495d83">Initialise the ODK XLSForm</span></h1>

In [2]:
path = "XLSForm_test.xlsx"
form_title = "ePoct+"
form_id = "epoct"
languages = ["English (en)", "French (fr)", "Swahili (sw)"]
def_language = "English"
version = "0.1"

In [3]:
ms = calgo.ClinicalAlgorithm()
ms.initODKExport(form_title, form_id, version, languages, def_language)
titles = [
    {"label":"Tools for Integrated Management of Childhood Illnesses", "level":1, "color":"#cae3ee"},
    {"label":"Clinical Decision Support Algorithm", "level":2, "color":"#495d83"}
]
ms.addWelcomeScreen(titles)

# Define variables

## Multiple answer choices

In [4]:
choices = {
    "NB_OF_CONVULSIONS":["1 convulsion","2 or more convulsions"],
    "PALMAR_PALLOR":["None", "Some", "Severe"]}
ms.addAnswers(choices)

## Questions

### Registration questions

In [5]:
ms.addRegistrationQuestions()

### Symptoms

In [6]:
t = "symptom"
symptoms = [
    {"label":"History of convulsions", "type":t, "asw_lbl":"YESNO"},
    {"label":"Number of convulsions", "type":t, "asw_lbl":"NB_OF_CONVULSIONS"},
    {"label":"Cough", "type":t, "asw_lbl":"YESNO"},
    {"label":"Difficulty breathing", "type":t, "asw_lbl":"YESNO"},
    {"label":"History of fever", "type":t, "asw_lbl":"YESNO"},
    {"label":"Diarrhoea", "type":t, "asw_lbl":"YESNO"},
    {"label":"Runny nose", "type":t, "asw_lbl":"YESNO"},
    {"label":"Ear problem", "type":t, "asw_lbl":"YESNO"}
]
ms.addQuestions(symptoms)

### Observed signs

In [7]:
t = "observed sign"
observed_signs = [
    {"label":"Unconscious", "type":t, "asw_lbl":"YESNO"},
    {"label":"Lethargic", "type":t, "asw_lbl":"YESNO"},
    {"label":"Convulsing now", "type":t, "asw_lbl":"YESNO"},
    {"label":"Significant haemoptysis", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Possible inhalation / swallowing of foreign object", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Barking cough", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Grunting", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Central cyanosis", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Wheeze", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Stridor", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Stridor at rest", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Lower chest indrawing", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Mouth lesion", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Epistaxis", "type":t, "asw_lbl":"YESNO_NA", "default":2},
    {"label":"Unable to talk /complete sentences", "type":t, "asw_lbl":"YESNO_NA", "default":2}
]
ms.addQuestions(observed_signs)

### Time duration

In [8]:
t = "time duration"
tduration = [
    {"label":"Number of days since onset of respiratory symptoms", "type":t, "unit":"days","constr":[1,20]},
    {"label":"Duration of fever", "type":t, "unit":"days","constr":[1,20]}
]
ms.addQuestions(tduration)

### Quantities

In [9]:
t = "quantity"
quantity = [
    {"label":"Number of loose or liquid stools over the last 24 hours", "type":t, "unit":"","constr":[1,5]}
]
ms.addQuestions(quantity)

### Measurements

In [10]:
t = "measurement"
measurements = [
    {
        "label":"Temperature",
        "odk_type":"range",
        "type":t,
        "unit":"degree Celsius (°C)",
        "parameters":[34,42,0.5]},
    {
        "label":"Weight",
        "odk_type":"integer",
        "type":t,
        "unit":"kg",
        "constr":[0,60]},
    {
        "label":"Respiratory rate",
        "odk_type":"integer",
        "type":t,
        "unit":"beats / minutes",
        "constr":[0,100]},
    {
        "label":"SpO2",
        "odk_type":"integer",
        "type":t,
        "unit":"%",
        "constr":[80,100]},
    {
        "label":"Pulse rate",
        "odk_type":"integer",
        "type":t,
        "unit":"beats / minutes",
        "constr":[40,200]}
]
ms.addQuestions(measurements)

### Calculated variables

In [11]:
t = "boolean"
boolean = [
    {"label":"Young infant",
     "type":t,
     "condition_list":[
         {'cond':calc.InfEqual2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 2).get()}],
     "istrue":1,
     "isfalse":0}
]
ms.addQuestions(boolean)

boolean = [
    {"label":"Fever",
     "type":t,
     "condition_list":[
         {'cond':calc.SupEqual2ThresCond(ms.getQuestions().getItem("Temperature"), 38).get()},
         {'cond':calc.EqualAnswerCond(ms.getQuestions().getItem("History of fever"), "Yes").get(),'logic':"or", 'group':0}
     ],
     "istrue":1,
     "isfalse":0},
]
ms.addQuestions(boolean)

booleans = [
    {"type":"boolean",
     "label":"Hypoxaemia",
     "condition_list":[
         {'cond':calc.Inf2ThresCond(ms.getQuestions().getItem("SpO2"), 90).get()}],
     "istrue":1,
     "isfalse":0},
    {"type":"boolean",
     "label":"Respiratory symptoms < 2 weeks",
     "condition_list":[
         {'cond':calc.InfEqual2ThresCond(ms.getQuestions().getItem("Number of days since onset of respiratory symptoms"), 14).get()}],
     "istrue":1,
     "isfalse":0},
    {"type":"boolean",
     "label":"Fast breathing",
     "condition_list":[
         {'cond':calc.SupEqual2ThresCond(ms.getQuestions().getItem("Respiratory rate"), 50).get()},
         {'cond':calc.Sup2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 2).get(), "logic": "and", "group":0},
         {'cond':calc.Inf2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 12).get(), "logic": "and", "group":0},
         {'cond':calc.SupEqual2ThresCond(ms.getQuestions().getItem("Respiratory rate"), 40).get(), "logic": "or", "group":1},
         {'cond':calc.SupEqual2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 12).get(), "logic": "and", "group":1}],
     "istrue":1,
     "isfalse":0},
    {"type":"boolean",
     "label":"Fast pulse rate",
     "condition_list":[
         {'cond':calc.Sup2ThresCond(ms.getQuestions().getItem("Pulse rate"), 180).get()},
         {'cond':calc.Sup2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 36).get(), "logic": "and", "group":0},
         {'cond':calc.Sup2ThresCond(ms.getQuestions().getItem("Pulse rate"), 200).get(), "logic": "or", "group":1},
         {'cond':calc.InfEqual2ThresCond(ms.getQuestions().getItem("Calculated age in months"), 36).get(), "logic": "and", "group":1}],
     "istrue":1,
     "isfalse":0},
]
ms.addQuestions(booleans)

## Diagnoses

### Severe diagnoses

In [12]:
severe_diagnoses = [
    "Danger signs",
    "Severe acute malnutrition", 
    "Complicated severe acute malnutrition",
    "Suspicion of meningitis",
    "Mastoiditis",
    "Severe pneumonia or very severe disease",
    "Possible severe asthma",
    "Suspicion of foreign object in airways",
    "Severe croup"
]
ms.addDiagnoses(severe_diagnoses, "severe")

### Moderate diagnoses

In [13]:
moderate_diagnoses = [
    "Complicated acute ear infection",
    "Complicated chronic ear infection",
    "Malaria",
    "Possible malaria",
    "Disentery",
    "Moderate malnutrition",
    "Pneumonia",
    "Possible moderate asthma",
    "Mild croup",
    "Hemoptysis"
]
ms.addDiagnoses(moderate_diagnoses, "moderate")

### Mild diagnoses

In [14]:
mild_diagnoses = [
    "Uncomplicated acute ear infection",
    "Chronic ear infection",
    "Acute diarrhoea",
    "Constipation",
    "Contusion",
    "Foreign body in ear",
    "Cough / cold",
    "Possible mild asthma"
]
ms.addDiagnoses(mild_diagnoses, "mild")

### Other diagnoses

In [15]:
other_diagnoses = [
    "Suspicion of tuberculosis",
]
ms.addDiagnoses(other_diagnoses, "other")

## Managements

In [16]:
managements = [
    "Urgent referral",
    "Non-urgent referral",
    "Admission / referral IF",
    "First line treatment",
    "Second line treatment",
]
ms.addManagements(managements)

## Treatments

In [17]:
treatments = [
    "Oxygen",
    "Treat for / prevent low blood sugar",
    "Keep warm",
    "Avoid agitation",
    "Rapid acting bronchodilator",
    "Soothe throat / relieve cough",
    "Return advice"
]
ms.addTreatments(treatments)

### Drugs

In [18]:
drugs = [
    "Benzylpenicillin",
    "Gentamicin",
    "Amoxicillin DT",
    "Prednisolone",
    "Vitamin A"
]
ms.addTreatments(drugs)

# Create modules

## Registration module

In [19]:
ms.addRegistrationModule("Registration")

## First look assessment module

### Danger signs

In [20]:
danger_sign_table_questions = [
    "Unconscious",
    "Lethargic",
    "Convulsing now",
    "History of convulsions"
]
danger_sign_other_questions = [
    "Number of convulsions"
]

In [21]:
mname = "First look assessment"
ms.addModule(mname, ms.getHeader()["Registration"])

smname = "Danger signs"
ms.addTableSubModule(smname, mname)
for k in danger_sign_table_questions:
    ms.getSubModule(smname).addQuestion(k)
    
smname = "Other danger signs"
ms.addSubModule(smname, mname)
for k in danger_sign_other_questions:
    ms.getSubModule(smname).addQuestion(k)

In [22]:
top_conditions = [
    {
        "child":"History of convulsions",
        "parent":"Convulsing now",
        "answer":"No"},
    {
        "child":"Number of convulsions",
        "parent":"History of convulsions",
        "answer":"Yes"}
]
ms.addQuestionTopConditions(top_conditions)

### Main symptoms

In [23]:
main_symptom_questions = [
    "Cough",
    "Difficulty breathing",
    "Number of days since onset of respiratory symptoms",
    "Diarrhoea",
    "History of fever",
    "Duration of fever",
    "Ear problem",
    "Runny nose"
]

In [24]:
smname = "Main symptoms"
ms.addTableSubModule(smname, mname)
for k in main_symptom_questions:
    ms.getSubModule(smname).addQuestion(k)

In [25]:
top_conditions = [
    {
        "child":"Number of days since onset of respiratory symptoms",
        "parent":"Cough",
        "answer":"Yes"},
    {
        "child":"Number of days since onset of respiratory symptoms",
        "parent":"Difficulty breathing",
        "answer":"Yes"},
    {
        "child":"Duration of fever",
        "parent":"History of fever",
        "answer":"Yes"}
]
ms.addQuestionTopConditions(top_conditions)

In [26]:
general_calculations = [
    "Fever"
]

In [27]:
ms.getModule(mname).addCalculationSubModule()
for k in general_calculations:
    ms.getModule(mname).getCalculationSubModules()[-1].addQuestion(k)

### Danger signs

In [28]:
criteria = [
    {
        "diagnosis":"Danger signs",
        "criterion":"Unconscious",
        "answer":"Yes"},
    {
        "diagnosis":"Danger signs",
        "criterion":"Lethargic",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Danger signs",
        "criterion":"Convulsing now",
        "answer":"Yes",
        "logic":"or",
        "group":1},
    {
        "diagnosis":"Danger signs",
        "criterion":"Number of convulsions",
        "answer":"2 or more convulsions",
        "logic":"or",
        "group":1}
    
]

ms.addDiagnosisCriteria(mname, criteria)

#### Possible malaria

In [29]:
criteria = [
    {
        "diagnosis":"Possible malaria",
        "criterion":"Fever",
        "answer":1}
]
ms.addDiagnosisCriteria(mname, criteria)

In [30]:
general_calculations = [
    "Danger signs",
    "Possible malaria"
]
ms.getModule(mname).addDiagnosisCalculationSubModule()
for k in general_calculations:
    ms.getModule(mname).getCalculationSubModules()[-1].addQuestion(ms.getDiagnoses().getItem(k).getLabel())
ms.getModule(mname).getCalculationSubModules()[-1].printQuestionList()

Danger signs
Possible malaria


## Basic measurement module

In [31]:
basic_measurement_questions = [
    "Temperature",
    "Weight"
]

In [32]:
mname = "Basic measurements"
ms.addModule(mname, ms.getHeader()["Registration"])
for k in basic_measurement_questions:
    ms.getModule(mname).addQuestion(k)

In [33]:
ms.addHeader()
ms.getModule(mname).addCalculationSubModule()
for k in ["CHeader"]:
    ms.getModule(mname).getCalculationSubModules()[-1].addQuestion(k)
ms.getModule(mname).getCalculationSubModules()[-1].printQuestionList()

CHeader


## Respiratory problem module

In [34]:
respiratory_symptom_sign_questions = [
    "Significant haemoptysis",
    "Possible inhalation / swallowing of foreign object",
    "Stridor",
    "Stridor at rest",
    "Barking cough",
    "Grunting",
    "Central cyanosis",
    "Wheeze",
    "Unable to talk /complete sentences"
]

respiratory_physical_exam_questions = [
    "Respiratory rate",
    "Lower chest indrawing",
    "SpO2",
    "Pulse rate"
]

In [35]:
mname = "Respiratory problems"
ms.addModule(mname, ms.getHeader()["Consultation"])
smname = "Symptoms and signs"
ms.addTableSubModule(smname, mname)
for k in respiratory_symptom_sign_questions:
    ms.getSubModule(smname).addQuestion(k)
smname = "Physical exam"
ms.addTableSubModule(smname, mname)
for k in respiratory_physical_exam_questions:
    ms.getSubModule(smname).addQuestion(k)

In [36]:
top_conditions = [
    {
        "child":"Respiratory problems",
        "parent":"Cough",
        "answer":"Yes"},
    {
        "child":"Respiratory problems",
        "parent":"Difficulty breathing",
        "answer":"Yes"}
    ]
ms.addModuleTopConditions(top_conditions)

top_conditions = [
    {
        "child":"Respiratory problems",
        "parent":"Danger signs",
        "answer":0,
        "logic":"and",
        "group":1}
    ]
ms.addModuleTopConditionDiagnoses(top_conditions)

In [37]:
top_conditions = [
    {
        "child":"Stridor at rest",
        "parent":"Stridor",
        "answer":"Yes"},
    {
        "child":"Mouth lesion",
        "parent":"Significant haemoptysis",
        "answer":"Yes"},
    {
        "child":"Epistaxis",
        "parent":"Significant haemoptysis",
        "answer":"Yes"},
    {
        "child":"Unable to talk /complete sentences",
        "parent":"Wheeze",
        "answer":"Yes"},
    {
        "child":"Pulse rate",
        "parent":"Wheeze",
        "answer":"Yes"},
    ]
ms.addQuestionTopConditions(top_conditions)

In [38]:
respiratory_calculations = [
    "Hypoxaemia",
    "Respiratory symptoms < 2 weeks",
    "Fast breathing",
    "Fast pulse rate"
]

In [39]:
ms.getModule(mname).addCalculationSubModule()
for k in respiratory_calculations:
    ms.getModule(mname).getCalculationSubModules()[-1].addQuestion(k)

#### Severe pneumonia or very severe disease

In [40]:
criteria = [
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "criterion":"Grunting",
        "answer":"Yes",
        "logic":"and",
        "group":1},
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "criterion":"Central cyanosis",
        "answer":"Yes",
        "logic":"or",
        "group":1},
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "criterion":"Hypoxaemia",
        "answer":1,
        "logic":"or",
        "group":1}
]
ms.addDiagnosisCriteria(mname, criteria)

criteria = [
    {
        "diagnosis":"Severe pneumonia or very severe disease",
        "including_diagnosis":"Danger signs",
        "logic":"or",
        "group":1}
]
ms.addIncludingDiagnosis(mname, criteria)

#### Pneumonia

In [41]:
criteria = [
    {
        "diagnosis":"Pneumonia",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Pneumonia",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Pneumonia",
        "criterion":"Lower chest indrawing",
        "answer":"Yes",
        "logic":"and",
        "group":1},
    {
        "diagnosis":"Pneumonia",
        "criterion":"Fast breathing",
        "answer":1,
        "logic":"or",
        "group":1}
]
ms.addDiagnosisCriteria(mname, criteria)

#### Cough / cold

In [42]:
criteria = [
    {
        "diagnosis":"Cough / cold",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Cough / cold",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0}
]
ms.addDiagnosisCriteria(mname, criteria)

#### Possible severe asthma

In [43]:
criteria = [
    {
        "diagnosis":"Possible severe asthma",
        "criterion":"Wheeze",
        "answer":"Yes"},
     {
        "diagnosis":"Possible severe asthma",
        "criterion":"Unable to talk /complete sentences",
        "answer":"Yes",
        "logic":"and",
        "group":1},
    {
        "diagnosis":"Possible severe asthma",
        "criterion":"Fast pulse rate",
        "answer":1,
        "logic":"or",
        "group":1}
]
ms.addDiagnosisCriteria(mname, criteria)

#### Possible moderate asthma

In [44]:
criteria = [
    {
        "diagnosis":"Possible moderate asthma",
        "criterion":"Wheeze",
        "answer":"Yes"},
    {
        "diagnosis":"Possible mild asthma",
        "criterion":"Wheeze",
        "answer":"Yes"},
    {
        "diagnosis":"Suspicion of foreign object in airways",
        "criterion":"Possible inhalation / swallowing of foreign object",
        "answer":"Yes"},
    {
        "diagnosis":"Severe croup",
        "criterion":"Stridor at rest",
        "answer":"Yes"},
    {
        "diagnosis":"Severe croup",
        "criterion":"Possible inhalation / swallowing of foreign object",
        "answer":"No",
        "logic":"and",
        "group":0}
]
ms.addDiagnosisCriteria(mname,criteria)

#### Mild croup

In [45]:
criteria = [
    {
        "diagnosis":"Mild croup",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Mild croup",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Mild croup",
        "criterion":"Stridor",
        "answer":"Yes",
        "logic":"and",
        "group":1},
    {
        "diagnosis":"Mild croup",
        "criterion":"Barking cough",
        "answer":"Yes",
        "logic":"or",
        "group":1}
]
ms.addDiagnosisCriteria(mname,criteria)

#### Hemoptysis

In [46]:
criteria = [
    {
        "diagnosis":"Hemoptysis",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Hemoptysis",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Hemoptysis",
        "criterion":"Respiratory symptoms < 2 weeks",
        "answer":1,
        "logic":"and",
        "group":1},
    {
        "diagnosis":"Hemoptysis",
        "criterion":"Significant haemoptysis",
        "answer":"Yes",
        "logic":"and",
        "group":2},
]
ms.addDiagnosisCriteria(mname,criteria)

#### Suspicion of TB

In [47]:
criteria = [
    {
        "diagnosis":"Suspicion of tuberculosis",
        "criterion":"Cough",
        "answer":"Yes"},
    {
        "diagnosis":"Suspicion of tuberculosis",
        "criterion":"Difficulty breathing",
        "answer":"Yes",
        "logic":"or",
        "group":0},
    {
        "diagnosis":"Suspicion of tuberculosis",
        "criterion":"Respiratory symptoms < 2 weeks",
        "answer":0,
        "logic":"and",
        "group":1}
]
ms.addDiagnosisCriteria(mname,criteria)

In [48]:
respiratory_diseases_calculations = [
    "Severe pneumonia or very severe disease",
    "Pneumonia",
    "Cough / cold",
    "Possible severe asthma",
    "Possible moderate asthma",
    "Possible mild asthma",
    "Suspicion of foreign object in airways",
    "Severe croup",
    "Mild croup",
    "Hemoptysis",
    "Suspicion of tuberculosis"
]
ms.getModule(mname).addDiagnosisCalculationSubModule()
for k in respiratory_diseases_calculations:
    ms.getModule(mname).getCalculationSubModules()[-1].addQuestion(ms.getDiagnoses().getItem(k).getLabel())

## Ear / Nose / Throat / Mouth module

In [49]:
ent_questions = [
    "Mouth lesion",
    "Epistaxis"
]

In [50]:
mname = "Ear / Nose / Throat / Mouth problems"
ms.addModule(mname, ms.getHeader()["Consultation"])

In [51]:
top_conditions = [
    {
        "child":"Ear / Nose / Throat / Mouth problems",
        "parent":"Ear problem",
        "answer":"Yes"},
    {
        "child":"Ear / Nose / Throat / Mouth problems",
        "parent":"Significant haemoptysis",
        "answer":"Yes",
        "logic":"or",
        "group":1}
    ]
ms.addModuleTopConditions(top_conditions)

In [52]:
smname = "Symptoms and signs"
ms.addTableSubModule(smname, mname)
for k in ent_questions:
    ms.getSubModule(smname).addQuestion(k)

## Skin module

## Gastro-intestinal problem module

In [53]:
mname = "Gastro-intestinal problems"
ms.addModule(mname, ms.getHeader()["Consultation"])
ms.getModule(mname).addQuestion("Number of loose or liquid stools over the last 24 hours")

In [54]:
top_conditions = [
    {
        "child":"Gastro-intestinal problems",
        "parent":"Diarrhoea",
        "answer":"Yes"}
    ]
ms.addModuleTopConditions(top_conditions)

In [55]:
top_conditions = [
    {
        "child":"Number of loose or liquid stools over the last 24 hours",
        "parent":"Diarrhoea",
        "answer":"Yes"}
    ]
ms.addQuestionTopConditions(top_conditions)

# Test assessment module

# Diagnosis module

In [56]:
excluding_diagnoses = [
    {
        "diagnosis":"Pneumonia",
        "excluding_diagnosis":"Severe pneumonia or very severe disease",
        "logic":"and",
        "group":51},
    {
        "diagnosis":"Danger signs",
        "excluding_diagnosis":"Severe pneumonia or very severe disease",
        "logic":"and",
        "group":51},
    {
        "diagnosis":"Cough / cold",
        "excluding_diagnosis":"Severe pneumonia or very severe disease",
        "logic":"and",
        "group":51},
    {
        "diagnosis":"Cough / cold",
        "excluding_diagnosis":"Pneumonia",
        "logic":"and",
        "group":52},
    {
        "diagnosis":"Possible mild asthma",
        "excluding_diagnosis":"Possible severe asthma",
        "logic":"and",
        "group":51},
    {
        "diagnosis":"Possible moderate asthma",
        "excluding_diagnosis":"Possible severe asthma",
        "logic":"and",
        "group":51},
     {
        "diagnosis":"Possible mild asthma",
        "excluding_diagnosis":"Possible moderate asthma",
        "logic":"and",
        "group":52},
    {
        "diagnosis":"Mild croup",
        "excluding_diagnosis":"Severe croup",
        "logic":"and",
        "group":51}
]

In [57]:
mname = "Diagnosis calculations"
ms.addDiagnosisCalculations(mname)
ms.addExcludingDiagnosis(mname,excluding_diagnoses)
for k in ms.getDiagnoses().get().keys():
    ms.getModule(mname).addQuestion(k)

mname = "Diagnoses"
ms.addDiagnosisModule(mname)

<h1><span style="color:#495d83">Management module</span></h1>

In [58]:
diagnosis_managements = [
    {
        "mgt":"Urgent referral",
        "diagnosis":"Danger signs"},
    {
        "mgt":"Urgent referral",
        "diagnosis":"Severe pneumonia or very severe disease"},
    {
        "mgt":"Admission / referral IF",
        "diagnosis":"Pneumonia"},
    {
        "mgt":"Urgent referral",
        "diagnosis":"Possible severe asthma"},
    {
        "mgt":"Admission / referral IF",
        "diagnosis":"Possible moderate asthma"},
    {
        "mgt":"First line treatment",
        "diagnosis":"Possible mild asthma"},
    {
        "mgt":"Urgent referral",
        "diagnosis":"Suspicion of foreign object in airways"},
    {
        "mgt":"Urgent referral",
        "diagnosis":"Severe croup"},
    {
        "mgt":"First line treatment",
        "diagnosis":"Mild croup"},
    {
        "mgt":"Second line treatment",
        "diagnosis":"Mild croup"},
    {
        "mgt":"Non-urgent referral",
        "diagnosis":"Mild croup"},
    {
        "mgt":"Admission / referral IF",
        "diagnosis":"Possible malaria"}
]
ms.addDiagnosisManagement(mname, diagnosis_managements)

In [59]:
excluding_managements = [
    {
        "mgt":"Non-urgent referral",
        "excluding_mgt":"Urgent referral",
        "logic":"and",
        "group":11},
    {
        "mgt":"Admission / referral IF",
        "excluding_mgt":"Urgent referral",
        "logic":"and",
        "group":11},
    {
        "mgt":"Second line treatment",
        "excluding_mgt":"First line treatment",
        "logic":"and",
        "group":11},
]

In [60]:
mname = "Management calculations"
ms.addManagementCalculations(mname)
ms.addExcludingManagement(mname,excluding_managements)
for k in ms.getManagements().get().keys():
    ms.getModule(mname).addQuestion(k)

In [61]:
top_conditions = [
    {
        "child":"Admission / referral IF",
        "parent":"Urgent referral",
        "answer":"Not possible",
        "logic":"or",
        "group":0},
    {
        "child":"First line treatment",
        "parent":"Admission / referral IF",
        "answer":"Not possible",
        "logic":"or",
        "group":0},
    {
        "child":"Second line treatment",
        "parent":"First line treatment",
        "answer":"Not possible",
        "logic":"or",
        "group":0}
    ]
ms.addManagementTopConditions(top_conditions)

In [62]:
mname = "Management"
ms.addManagementModule(mname)

<h1><span style="color:#495d83">Treatment module</span></h1>

<h2><span style="color:#ed8a78">Oxygen</span></h2>

In [63]:
treatments = [
    {
        "treatment":"Oxygen",
        "criterion":"Hypoxaemia",
        "answer":1}
]
ms.addTreatment2Criterion(mname, treatments)

<h2><span style="color:#ed8a78">Severe pneumonia or very severe disease</span></h2>

In [64]:
treatments = [
    {
        "treatment":"Benzylpenicillin",
        "diagnosis":"Severe pneumonia or very severe disease"},
    {
        "treatment":"Gentamicin",
        "diagnosis":"Severe pneumonia or very severe disease"},
    {
        "treatment":"Treat for / prevent low blood sugar",
        "diagnosis":"Severe pneumonia or very severe disease"},
    {
        "treatment":"Keep warm",
        "diagnosis":"Severe pneumonia or very severe disease"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Pneumonia</span></h2>

In [65]:
treatments = [
    {
        "treatment":"Amoxicillin DT",
        "diagnosis":"Pneumonia"},
    {
        "treatment":"Vitamin A",
        "diagnosis":"Pneumonia"},
    {
        "treatment":"Soothe throat / relieve cough",
        "diagnosis":"Pneumonia"},
    {
        "treatment":"Return advice",
        "diagnosis":"Pneumonia"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Cough / cold</span></h2>

In [66]:
treatments = [
    {
        "treatment":"Soothe throat / relieve cough",
        "diagnosis":"Cough / cold"},
    {
        "treatment":"Return advice",
        "diagnosis":"Cough / cold"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Possible mild asthma</span></h2>

In [67]:
treatments = [
    {
        "treatment":"Soothe throat / relieve cough",
        "diagnosis":"Possible mild asthma"},
    {
        "treatment":"Return advice",
        "diagnosis":"Possible mild asthma"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Severe croup</span></h2>

In [68]:
treatments = [
    {
        "treatment":"Prednisolone",
        "diagnosis":"Severe croup",
        "dose":"2mg/kg"},
    {
        "treatment":"Avoid agitation",
        "diagnosis":"Severe croup"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Mild croup</span></h2>

In [69]:
treatments = [
    {
        "treatment":"Soothe throat / relieve cough",
        "diagnosis":"Mild croup"},
    {
        "treatment":"Return advice",
        "diagnosis":"Mild croup"}
]
ms.addTreatment2Diagnosis(mname, treatments)

<h2><span style="color:#ed8a78">Hemoptysis</span></h2>

In [70]:
treatments = [
    {
        "treatment":"Soothe throat / relieve cough",
        "diagnosis":"Hemoptysis"},
    {
        "treatment":"Return advice",
        "diagnosis":"Hemoptysis"}
]
ms.addTreatment2Diagnosis(mname, treatments)

In [71]:
mname = "Treatment calculations"
ms.addTreatmentCalculations(mname)
for k in ms.getTreatments().get().keys():
    ms.getModule(mname).addQuestion(k)

In [72]:
mname = "Treatments"
ms.addTreatmentModule(mname)

## Summary module

In [73]:
mname = "Summary"
ms.addModule(mname)
smname = "Main symptoms"
ms.addSubModule(smname, mname)
smname = "Diagnoses"
ms.addSubModule(smname, mname)
smname = "Managements"
ms.addSubModule(smname, mname)
smname = "Treatments"
ms.addSubModule(smname, mname)

In [74]:
ms.ODKExport(path)

Registration
<class 'clinical_algorithm.RegistrationModule'>
First look assessment
<class 'clinical_algorithm.ConsultationModule'>
Basic measurements
<class 'clinical_algorithm.ConsultationModule'>
Respiratory problems
<class 'clinical_algorithm.ConsultationModule'>
Ear / Nose / Throat / Mouth problems
<class 'clinical_algorithm.ConsultationModule'>
Gastro-intestinal problems
<class 'clinical_algorithm.ConsultationModule'>
Diagnosis calculations
<class 'clinical_algorithm.DiagnosisCalculationModule'>
Diagnoses
<class 'clinical_algorithm.Module'>
Management calculations
<class 'clinical_algorithm.ManagementCalculationModule'>
Management
<class 'clinical_algorithm.Module'>
Treatment calculations
<class 'clinical_algorithm.TreatmentCalculationModule'>
Treatments
<class 'clinical_algorithm.Module'>
Summary
<class 'clinical_algorithm.ConsultationModule'>
